In [1]:
from ipyparallel import Client
rc = Client(profile="mpi")

In [2]:
with rc[:].sync_imports():
    from mpi4py import MPI
    from numpy.random import rand, randn
    from skeletor import cppinit, Field, Float, Float2, Grid, Particles, Sources

importing MPI from mpi4py on engine(s)
importing rand,randn from numpy.random on engine(s)
importing cppinit,Field,Float,Float2,Grid,Particles,Sources from skeletor on engine(s)


In [3]:
%%px

def prepare1(nx=64, ny=64, npc=512, vtx=1.0, vty=1.0, comm=MPI.COMM_WORLD):

    idproc, nvp = cppinit(comm)

    grid = Grid(nx, ny, comm)

    npar = npc*nx*ny
    npmax = int(1.5*npar/nvp)
    ions = Particles(npmax)

    sources = Sources(grid, comm, dtype=Float)

    x = nx*rand(npar).astype(Float)
    y = ny*rand(npar).astype(Float)
    vx = vtx*randn(npar).astype(Float)
    vy = vty*randn(npar).astype(Float)

    ions.initialize(x, y, vx, vy, grid)
    
    sources.rho.fill(0.0)
    
    return ions, sources

In [4]:
%%timeit -n 100 %px ions, sources = prepare1()
%px sources.deposit(ions, erase=False)

100 loops, best of 3: 27.1 ms per loop


In [5]:
%%timeit -n 100 %px ions, sources = prepare1()
%px sources.deposit_ppic2(ions, erase=False)

100 loops, best of 3: 18.2 ms per loop


In [6]:
%%px

def prepare2(nx=1024, ny=1024, comm=MPI.COMM_WORLD):

    idproc, nvp = cppinit(comm)

    grid = Grid(nx, ny, comm)

    rho = Field(grid, comm, dtype=Float)
    rho[:,:] = randn(*rho.shape)
    
    return rho

In [7]:
%%timeit -n 100 %px rho = prepare2()
%px rho.add_guards()

100 loops, best of 3: 9.86 ms per loop


In [8]:
%%timeit -n 100 %px rho = prepare2()
%px rho.add_guards2()

100 loops, best of 3: 9.95 ms per loop


In [9]:
%%timeit -n 100 %px rho = prepare2()
%px rho.add_guards_ppic2()

100 loops, best of 3: 9.49 ms per loop


In [10]:
%%px

def prepare3(nx=1024, ny=1024, comm=MPI.COMM_WORLD):

    idproc, nvp = cppinit(comm)

    grid = Grid(nx, ny, comm)

    E = Field(grid, comm, dtype=Float2)

    E[:,:]["x"] = randn(*E.shape)
    E[:,:]["y"] = randn(*E.shape)
    
    return E

In [11]:
%%timeit -n 100 %px E = prepare3()
%px E.copy_guards()

100 loops, best of 3: 10.1 ms per loop


In [12]:
%%timeit -n 100 %px E = prepare3()
%px E.copy_guards2()

100 loops, best of 3: 10 ms per loop


In [13]:
%%timeit -n 100 %px E = prepare3()
%px E.copy_guards_ppic2()

100 loops, best of 3: 9.37 ms per loop
